In [1]:
# For OpenAI

import logging
import os
import sys

from llama_index.core import Settings
from llama_index.llms.openai import OpenAI

# Add OPENAI_API_KEY to your environment variables

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# define LLM
# NOTE: at the time of demo, text-davinci-002 did not have rate-limit errors
llm = OpenAI(temperature=0, model="gpt-3.5-turbo")

Settings.llm = llm
Settings.chunk_size = 512

In [2]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader
from llama_index.core import StorageContext
from llama_index.graph_stores.nebula import NebulaGraphStore


from llama_index.llms.openai import OpenAI
from IPython.display import Markdown, display

In [30]:
# DATA_FOLDER = os.getenv("DATA_FOLDER", "./data")
DATA_FOLDER = "./my_data"
documents = SimpleDirectoryReader(f"./{DATA_FOLDER}").load_data()

[Document(id_='aca01e65-807d-4ec9-bda7-599da138cae2', embedding=None, metadata={'file_path': '/Users/Vidy/playground/screenshot-graphy-boi/my_data/test1.txt', 'file_name': 'test1.txt', 'file_type': 'text/plain', 'file_size': 40, 'creation_date': '2024-04-03', 'last_modified_date': '2024-04-03'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='There is some text about Sicara in here\n', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='ae0ac302-03a6-40bb-bb4f-5c72caeb9b60', embedding=None, metadata={'file_path': '/Users/Vidy/playground/screenshot-graphy-boi/my_data/test2.txt', 'file_name': 'test2.txt', 'file_type': 'text/plain', 'file_size': 473

In [22]:
os.environ["NEBULA_USER"] = "root"
os.environ[
    "NEBULA_PASSWORD"
] = "nebula"  # replace with your password, by default it is "nebula"
os.environ[
    "NEBULA_ADDRESS"
] = "127.0.0.1:9669"  # assumed we have NebulaGraph 3.5.0 or newer installed locally

# Assume that the graph has already been created
# Create a NebulaGraph cluster with:
# NebulaGraph Docker Extension https://hub.docker.com/extensions/weygu/nebulagraph-dd-ext
# Create graph space by inputing commands in the NebulaGraph's console (in docker desktop):
# ADD HOSTS "storaged0":9779,"storaged1":9779,"storaged2":9779
# CREATE SPACE graphy_boi(vid_type=FIXED_STRING(256), partition_num=1, replica_factor=1);
# :sleep 10;
# USE graphy_boi;
# CREATE TAG entity(name string);
# CREATE EDGE relationship(relationship string);
# CREATE TAG INDEX entity_index ON entity(name(256));

space_name = "graphy_boi"
edge_types, rel_prop_names = (
    ["relationship"],
    ["relationship"],
)  # default, could be omit if create from an empty kg
tags = ["entity"]  # default, could be omit if create from an empty kg

In [31]:
graph_store = NebulaGraphStore(
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)

# NOTE: can take a while!
index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=2,
    space_name=space_name,
    edge_types=edge_types,
    rel_prop_names=rel_prop_names,
    tags=tags,
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [35]:
query_engine = index.as_query_engine()

response = query_engine.query("Tell me more about Uber")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [26]:
response

Response(response="I'm sorry, I cannot provide information on Uber as there is no relevant context information provided.", source_nodes=[NodeWithScore(node=TextNode(id_='a51a5127-3eaf-47f4-9b16-63e759c9d7d7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='No relationships found.', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=1.0)], metadata={'a51a5127-3eaf-47f4-9b16-63e759c9d7d7': {}})

In [27]:
display(Markdown(f"<b>{response}</b>"))

<b>I'm sorry, I cannot provide information on Uber as there is no relevant context information provided.</b>

In [9]:
response = query_engine.query(
    "Tell me more about what the author worked on at Interleaf"
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No relationships found, returning nodes found by keywords.
INFO:llama_index.core.indices.knowledge_graph.retrievers:> No nodes found by keywords, returning empty response.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
%load_ext ngql

In [28]:
%ngql --address 127.0.0.1 --port 9669 --user root --password nebula

Connection Pool Created
INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,Name
0,graphy_boi


In [29]:
%%ngql
USE graphy_boi;
MATCH p=(n)-[*1..2]-()
  WHERE id(n) IN ['Interleaf', 'history', 'Software', 'Company']
RETURN p LIMIT 100;

INFO:nebula3.logger:Get connection to ('127.0.0.1', 9669)


,p


In [16]:
%ng_draw

<class 'pyvis.network.Network'> |N|=0 |E|=0